In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = zip(lats, lngs)
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

771

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    # Log the URL, record, and set numbers and the city.
    ##print(f"Processing Record {record_count} of Set {set_count} | {city} | {city_url}")
    # Add 1 to the record count.
    record_count += 1
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_desc = city_weather ["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_desc,
                          "Date": city_date})
# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
-----------------------------
Data Retrieval Complete      
-----------------------------


In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Souillac,MU,-20.5167,57.5167,78.80,94,75,13.80,light rain,2021-01-11 04:37:33
1,Tasiilaq,GL,65.6145,-37.6368,19.40,63,90,3.44,overcast clouds,2021-01-11 04:37:42
2,Mar Del Plata,AR,-38.0023,-57.5575,64.40,94,90,6.91,heavy intensity rain,2021-01-11 04:37:46
3,East London,ZA,-33.0153,27.9116,66.20,94,90,2.30,light rain,2021-01-11 04:35:31
4,Fortuna,US,40.5982,-124.1573,50.00,100,1,3.65,clear sky,2021-01-11 04:38:49
5,Klaksvik,FO,62.2266,-6.5890,30.20,80,75,14.97,light snow,2021-01-11 04:38:19
6,Ushuaia,AR,-54.8000,-68.3000,41.00,75,75,20.71,broken clouds,2021-01-11 04:32:42
7,Yellowknife,CA,62.4560,-114.3525,8.60,92,90,5.75,overcast clouds,2021-01-11 04:37:29
8,Rikitea,PF,-23.1203,-134.9692,78.24,79,35,18.03,scattered clouds,2021-01-11 04:35:20
9,Albany,US,42.6001,-73.9662,23.00,92,94,2.17,overcast clouds,2021-01-11 04:33:19


In [7]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")